# Import member pages 

In [ ]:
import pymongo 
import sys 
sys.path.insert(0,'../..')
import main

In [1]:
import scraper
import inspect

[autoreload of tornado.concurrent failed: Traceback (most recent call last):
  File "/Users/User1/anaconda/envs/python2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name is_finalizing
]
[autoreload of ipykernel.kernelbase failed: Traceback (most recent call last):
  File "/Users/User1/anaconda/envs/python2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name py3compat
]
[autoreload of ipykernel.kernelapp failed: Traceback (most recent call last):
  File "/Users/User1/anaconda/envs/python2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name ioloop
]
[autoreload of ipykernel.zmqshell failed: Traceback (most recent call last):
  File "/Users/User1/anaconda/envs/python2/lib

In [101]:
# Get list of methods in `scraper`
[a[0] for a in inspect.getmembers(scraper.Scraper, inspect.ismethod)]

['__init__',
 'get_community_members',
 'get_community_members_continue',
 'get_community_page_scrolled',
 'get_member_pages',
 'get_page_from_list',
 'use_firefox',
 'use_phantom']

In [102]:
# Get list of methods in `Parser`
[a[0] for a in inspect.getmembers(scraper.Parser, inspect.ismethod)]

['__init__',
 'check_member_records',
 'get_aboutme_data',
 'get_favorites_data',
 'get_followers_data',
 'get_following_data',
 'get_html',
 'get_madeits_data',
 'get_member_list_to_parse',
 'get_member_parsed_data',
 'get_members_aboutme_from_pkl',
 'get_members_favorites',
 'get_members_followers',
 'get_members_following',
 'get_members_madeits',
 'get_members_reviews',
 'get_recipe_data',
 'get_recipe_pkl_page_from_aws',
 'get_reviews_data']

In [103]:
# Create method `get_member_pages_from_S3` 

In [104]:
import pymongo 
import boto3
import os 
import json
AWS_KEY = os.environ['AWS_ACCESS_KEY']
AWS_SECRET = os.environ['AWS_SECRET_ACCESS_KEY'] 

In [22]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
allrecipes_db = client.allrecipes
member_pages_coll = allrecipes_db.member_pages
about_coll = allrecipes_db.about


# Create index based on member_ID for member_pages_coll 
``` 
member_pages_coll.create_index([('member_ID',pymongo.ASCENDING)], unique=True)
```

In [23]:
 # Open s3 session with boto3 and get files in 'member_pages/'
session = boto3.Session(aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
s3 = session.resource('s3')

mybucket = s3.Bucket('ohailolcat')
# Gets all objects passed through filter
member_page_files = [a.key for a in mybucket.objects.filter(Prefix='member_pages/')]

In [26]:
print len(member_page_files)

521

In [28]:
member_page_files[0]

u'member_pages/10090853_pages.pkl'

In [47]:
# These are cPickle files 
import cPickle

In [74]:
# Create an iterable of filtered ObjectSummary resources
mybucket.objects.filter(Prefix='member_pages/')

s3.Bucket.objectsCollection(s3.Bucket(name='ohailolcat'), s3.ObjectSummary)

In [129]:
# Collect all objectsummaries
memberfilesummary = [a for a in mybucket.objects.filter(Prefix='member_pages/')]

In [130]:
memberfilesummary[0]

s3.ObjectSummary(bucket_name='ohailolcat', key=u'member_pages/10090853_pages.pkl')

In [131]:
# Read the 'Body' of object
memfile = memberfilesummary[0].get()['Body'].read()

In [132]:
# Unpickle the string 
pklfile = cPickle.loads(memfile)

In [133]:
pklfile.keys()

['favorites_page',
 'reviews_page',
 'followers_page',
 'madeit_page',
 'member_ID',
 'recipes_page',
 'following_page',
 'aboutme_page']

In [134]:
# compare with member_pages_coll document contents
member_pages_coll.find_one().keys()

[u'favorites_page',
 u'following_page',
 u'reviews_page',
 u'aboutme_page',
 u'madeit_page',
 u'link',
 u'member_ID',
 u'followers_page',
 u'_id',
 u'recipes_page']

In [135]:
# All the keys are the same except `link` and `_id`, which is okay.  

def get_member_pages_from_s3(memlist=None): 
    """Given a list of member_IDs, populate member_pages_coll with data from files in 
    member_pages/"""
    if not memlist: 
        memlist = [a.key for a in mybucket.objects.filter(Prefix='member_pages/')]
    print "number of files in s3 bucket:", len(memlist)
    print "number of docs in member_pages_coll", member_pages_coll.count()
    member_pages_memlist = [a['member_ID'] for a in member_pages_coll.find()]
    # member_pages_coll.ensureIndex({name: 1, nodes: 1}, {unique: true, dropDups: true})
    # member_pages_coll.ensure_index({"member_ID": 1}, {unique:}) .... 
    # to_get = set(memlist) - set(member_pages_memlist)
    # print "files to get:", len(to_get)
    for i,a in enumerate(mybucket.objects.filter(Prefix='member_pages/')): 
        memdata = cPickle.loads(a.get()['Body'].read())
        if memdata["member_ID"] not in member_pages_memlist:
            print "{} Inserting: {}".format(i, memdata["member_ID"])
            member_pages_coll.insert_one(memdata)

In [136]:
get_member_pages_from_s3()

number of files in s3 bucket: 611
number of docs in member_pages_coll 1340
485 Inserting: 79shale97
522 Inserting: alanjswan
523 Inserting: alleyworld
524 Inserting: arcally
525 Inserting: bakingnana
526 Inserting: bdweld
527 Inserting: bookmarker
528 Inserting: celeste
529 Inserting: chefbourne
530 Inserting: chrisdukes
531 Inserting: cookingsweetheat
532 Inserting: crystalmeth
533 Inserting: cynth
534 Inserting: dabratcarol
535 Inserting: daddio-on-patio
536 Inserting: debbiedoo
537 Inserting: dianea
538 Inserting: dluttmann
539 Inserting: docsimonson
540 Inserting: duboo
541 Inserting: dwfalls78
542 Inserting: eake
543 Inserting: elizabeth
544 Inserting: emerggirl
545 Inserting: eyeluv2katz
546 Inserting: fitgal318
547 Inserting: foodaholic
548 Inserting: frackfamily5
549 Inserting: gails
550 Inserting: gottaloveit
551 Inserting: gourmetshand
552 Inserting: grumpygrandpa
553 Inserting: hansonmansion
554 Inserting: iceemama
555 Inserting: jhaan
556 Inserting: josephineroeper
557 Inse

In [127]:
docswolink = list(member_pages_coll.find({'aboutme_page':{"$exists": False}}))

In [128]:
len(docswolink)

0

In [111]:
docswolink_ids = [a['_id'] for a in docswolink]

In [137]:
member_pages_coll.count()

1430

In [ ]:
m